In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("cleaned_data.csv")
df.head()

C:\Users\shrey\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,original text,cleaned text,joke
0,0,I am soooo glad I'm not circumcised! My corona...,soooo glad circumcised corona covered foreskin...,1.0
1,1,Did you know Google now has a platform for rec...,know google platform recording bowel movements...,1.0
2,2,What is the difference between my wife and my ...,difference wife vacuum vacuum nt snore sex amp...,1.0
3,3,What did the French man say to the attractive ...,french man say attractive teletubby oo lala,1.0
4,4,"Yo Mama Yo momma's so fat, that when she went ...",yo mama yo momma fat went zoo hippos got jealous,1.0


In [4]:
df = df.drop(columns=df.columns.values[0:2])

In [5]:
df = df.dropna()

In [6]:
df_jokes = df[df['joke'] > 0]
df_non_jokes = df[df['joke'] < 0]

In [7]:
df = pd.concat([df_jokes.sample(25000, random_state=42),
                df_non_jokes.sample(25000, random_state=42)])
df.head()

,cleaned text,joke
544835,walked bar saw fat bird dancing table went sai...,1.0
120452,band antivaxxers refuse listen cure,1.0
100104,call building got engaged premrs,1.0
501226,shit gross one best friend dad used tell us me...,1.0
752947,hear couple stole calendar got six months,1.0


In [8]:
df['joke'].sum()

0.0

In [9]:
X = df.loc[:, 'cleaned text']
y = df.loc[:, 'joke']

# Split Data into Train and Test

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=42)

In [12]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

40000 40000
10000 10000


In [13]:
X_test[:5]

821108    apple google two brands world valued billion a...
102979    man feel embarrassed twice life first time get...
604087       wish rant sub eligible make front page deleted
425613    people always told make little things count de...
800317    official says us citizen detained north korea ...
Name: cleaned text, dtype: object

In [14]:
y_test[:5]

821108   -1.0
102979    1.0
604087    1.0
425613    1.0
800317   -1.0
Name: joke, dtype: float64

# Tfidf Vectorization

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
tfidf = TfidfVectorizer(ngram_range=(1, 2))

In [44]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.fit_transform(X_test)

# Dimensionality Reduction

## Chi2

In [45]:
from sklearn.feature_selection import chi2, SelectKBest

In [46]:
chi_200 = SelectKBest(chi2, k=200)
chi_500 = SelectKBest(chi2, k=500)

In [48]:
X_train_tfidf_200_chi = chi_200.fit_transform(X_train_tfidf, y_train)
X_train_tfidf_500_chi = chi_500.fit_transform(X_train_tfidf, y_train)
X_test_tfidf_200_chi = chi_200.fit_transform(X_test_tfidf, y_test)
X_test_tfidf_500_chi = chi_500.fit_transform(X_test_tfidf, y_test)

In [61]:
print(X_train_tfidf_200_chi.shape)
print(X_train_tfidf_500_chi.shape)
print(X_test_tfidf_200_chi.shape)
print(X_test_tfidf_500_chi.shape)

(40000, 200)
(40000, 500)
(10000, 200)
(10000, 500)


## Saving the files to CSV

In [55]:
# Gives error cause its a csr matrix, delete the code in this block and continue to model selection
np.savetxt(fname='data/chi-200-train.csv', X=X_train_tfidf_200_chi, delimiter=',')
np.savetxt('data/chi-500-train.csv', X_train_tfidf_500_chi, delimiter=',')
np.savetxt('data/chi-200-test.csv', X_test_tfidf_200_chi, delimiter=',')
np.savetxt('data/chi-500-test.csv', X_test_tfidf_500_chi, delimiter=',')

ValueError: Expected 1D or 2D array, got 0D array instead